In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd

In [3]:
#importing companies data and named it company
companies = pd.read_csv(r"E:\upgrad\intro to data management\project 1\companies.txt", sep = "\t",encoding = "ANSI")
companies.head()


,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
3,/Organization/0-6-Com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
4,/Organization/004-Technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010


In [4]:
#encoding special character from permalink and name
companies['permalink'] = companies.permalink.str.encode('utf-8').str.decode('ascii','ignore')
companies['name'] = companies.permalink.str.encode('utf-8').str.decode('ascii','ignore')

In [5]:
#importing rounds2 data and naming it rounds2
rounds2 = pd.read_csv(r"E:\upgrad\intro to data management\project 1\rounds2.csv",sep = ",",encoding ="ANSI")
rounds2.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


In [6]:
#encoding special character from company_permalink
rounds2['company_permalink'] = rounds2.company_permalink.str.encode('utf-8').str.decode('ascii','ignore')

In [7]:
#inspecting dataframe companies
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66368 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null object
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51147 non-null object
dtypes: object(10)
memory usage: 5.1+ MB


In [8]:
#inspecting rounds2 dataframe
rounds2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114949 entries, 0 to 114948
Data columns (total 6 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null object
funding_round_code         31140 non-null object
funded_at                  114949 non-null object
raised_amount_usd          94959 non-null float64
dtypes: float64(1), object(5)
memory usage: 5.3+ MB


In [9]:
#column wise null percentage in companies dataframe
round(100*(companies.isnull().sum()/len(companies.index)),2)

permalink         0.00
name              0.00
homepage_url      7.62
category_list     4.74
status            0.00
country_code     10.48
state_code       12.88
region           12.10
city             12.10
founded_at       22.93
dtype: float64

In [10]:
#dropping columns from companies dataframe
companies = companies.drop(['state_code','region','city','founded_at'],axis = 1)

In [11]:
#column wise null percentage in rounds2
round(100*(rounds2.isnull().sum()/len(rounds2.index)),2)

company_permalink           0.00
funding_round_permalink     0.00
funding_round_type          0.00
funding_round_code         72.91
funded_at                   0.00
raised_amount_usd          17.39
dtype: float64

In [12]:
#dropping column from rounds2 dataframe
rounds2 = rounds2.drop(['funding_round_code','funded_at'],axis = 1)

In [13]:
#changing permalink to upper case
companies['permalink'] = companies['permalink'].str.upper()

In [14]:
#changing company_permalink to upper case
rounds2['company_permalink'] = rounds2['company_permalink'].str.upper()

In [15]:
#dropping duplicates value from companies and getting unique values
companies = companies.drop_duplicates()
len(companies['permalink'].unique().tolist())

66368

In [16]:
#dropping duplicates values from rounds2 and getting unique values
rounds2 = rounds2.drop_duplicates()
len(rounds2['company_permalink'].unique().tolist())

66368

In [17]:
#calculating common values from both dataframe
len(set(companies.permalink).difference(rounds2.company_permalink))

0

In [18]:
#merging both dataframe and naming it master_frame
master_frame = pd.merge(companies,rounds2,right_on = 'company_permalink',left_on= 'permalink',how ='left')
master_frame.shape

(114949, 10)

In [19]:
#dropping rows from raised_amount_usd of null values
master_frame = master_frame[~np.isnan(master_frame['raised_amount_usd'])] 

In [20]:
#calculating funding round type in million dollar from master_frame
funding_by_round = master_frame.pivot_table(values = 'raised_amount_usd',index = 'funding_round_type',aggfunc = 'mean')
funding_by_round['raised_amount_usd'] = funding_by_round['raised_amount_usd'].apply(lambda x:(x/1000000))
funding_by_round['raised_amount_usd'] = funding_by_round['raised_amount_usd'].apply(lambda x:round(x,2))
funding_by_round  = funding_by_round.loc[['venture','angel','seed','private_equity']].sort_values(by= 'raised_amount_usd',ascending = False)
funding_by_round

,raised_amount_usd
funding_round_type,
private_equity,73.31
venture,11.75
angel,0.96
seed,0.72


In [21]:
#calculating top9 countries of funding type venture
top9 = master_frame.loc[(master_frame.funding_round_type == 'venture')]
top9 = top9.pivot_table(values = 'raised_amount_usd',index = 'country_code',aggfunc = 'sum').sort_values(by = 'raised_amount_usd',ascending = False)[:9]
top9 = top9.apply(lambda x:(x/1000000))
top9 = top9.apply(lambda x:round(x,2))
top9

,raised_amount_usd
country_code,
USA,422510.84
CHN,39835.42
GBR,20245.63
IND,14391.86
CAN,9583.33
FRA,7259.54
ISR,6907.51
DEU,6346.96
JPN,3363.68


In [22]:
#importing mapping files and naming it to mapping
mapping = pd.read_csv(r"E:\upgrad\intro to data management\project 1\mapping.csv")

In [23]:
#removing null value from mapping file of col category list and replacing 0 with na and in lower case
mapping = mapping[mapping.category_list.notnull()]
mapping['category_list'] = mapping['category_list'].apply(lambda x: x.replace('0','na')if ('2.0' not in x)else x)
mapping['category_list'] = mapping['category_list'].str.lower()
mapping

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
1,3d,0,0,0,0,0,1,0,0,0
2,3d printing,0,0,0,0,0,1,0,0,0
3,3d technology,0,0,0,0,0,1,0,0,0
4,accounting,0,0,0,0,0,0,0,0,1
5,active lifestyle,0,0,0,0,1,0,0,0,0
6,ad targeting,0,0,0,0,0,0,0,0,1
7,advanced materials,0,0,0,0,0,1,0,0,0
8,adventure travel,1,0,0,0,0,0,0,0,0
9,advertising,0,0,0,0,0,0,0,0,1
10,advertising exchanges,0,0,0,0,0,0,0,0,1


In [24]:
#primary_sector to lower case
master_frame['primary_sector'] = master_frame['category_list'].str.split('|',1).str[0]
master_frame['primary_sector'] = master_frame['primary_sector'].str.lower()

In [25]:
#extracting and mapping the main sector and primary sector from mapping
melt_data = pd.melt(mapping,id_vars = ['category_list'],var_name = 'Sectors',value_name = 'Main_sectors')

In [26]:
main_sector = melt_data.loc[melt_data.Main_sectors!=0]
master_frame['main_sector'] = master_frame.primary_sector.map(main_sector.set_index('category_list').Sectors)
master_frame

,permalink,name,homepage_url,category_list,status,country_code,company_permalink,funding_round_permalink,funding_round_type,raised_amount_usd,primary_sector,main_sector
0,/ORGANIZATION/-FAME,/Organization/-Fame,http://livfame.com,Media,operating,IND,/ORGANIZATION/-FAME,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,10000000.0,media,Entertainment
2,/ORGANIZATION/-QOUNTER,/Organization/-Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,/ORGANIZATION/-QOUNTER,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,700000.0,application platforms,"News, Search and Messaging"
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/Organization/-The-One-Of-Them-Inc-,http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,3406878.0,apps,"News, Search and Messaging"
4,/ORGANIZATION/0-6-COM,/Organization/0-6-Com,http://www.0-6.com,Curated Web,operating,CHN,/ORGANIZATION/0-6-COM,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,2000000.0,curated web,"News, Search and Messaging"
6,/ORGANIZATION/01GAMES-TECHNOLOGY,/Organization/01Games-Technology,http://www.01games.hk/,Games,operating,HKG,/ORGANIZATION/01GAMES-TECHNOLOGY,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,41250.0,games,Entertainment
7,/ORGANIZATION/0NDINE-BIOMEDICAL-INC,/Organization/0Ndine-Biomedical-Inc,http://ondinebio.com,Biotechnology,operating,CAN,/ORGANIZATION/0NDINE-BIOMEDICAL-INC,/funding-round/2b9d3ac293d5cdccbecff5c8cb0f327d,seed,43360.0,biotechnology,Cleantech / Semiconductors
8,/ORGANIZATION/0NDINE-BIOMEDICAL-INC,/Organization/0Ndine-Biomedical-Inc,http://ondinebio.com,Biotechnology,operating,CAN,/ORGANIZATION/0NDINE-BIOMEDICAL-INC,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,719491.0,biotechnology,Cleantech / Semiconductors
9,/ORGANIZATION/0XDATA,/Organization/0Xdata,http://h2o.ai/,Analytics,operating,USA,/ORGANIZATION/0XDATA,/funding-round/383a9bd2c04f7038bb543ccef5ba3eae,seed,3000000.0,analytics,"Social, Finance, Analytics, Advertising"
10,/ORGANIZATION/0XDATA,/Organization/0Xdata,http://h2o.ai/,Analytics,operating,USA,/ORGANIZATION/0XDATA,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,20000000.0,analytics,"Social, Finance, Analytics, Advertising"
11,/ORGANIZATION/0XDATA,/Organization/0Xdata,http://h2o.ai/,Analytics,operating,USA,/ORGANIZATION/0XDATA,/funding-round/ae2a174c06517c2394aed45006322a7e,venture,1700000.0,analytics,"Social, Finance, Analytics, Advertising"


In [27]:
#creating a new data frame D1 
D1 = master_frame.loc[(master_frame.funding_round_type == 'venture') & (master_frame.country_code == 'USA') &(master_frame.raised_amount_usd >= 5000000) & (master_frame.raised_amount_usd <= 15000000)]
D1['raised_amount_usd'] = D1['raised_amount_usd'].apply(lambda x: x/1000000)
D1['raised_amount_usd'] = D1['raised_amount_usd'].apply(lambda x: round(x,2))

In [28]:
#now calculating sum of investment and no of investment in main sectors and renaming col
D1_count_investment_sector = D1.pivot_table(values = 'raised_amount_usd',index = 'main_sector',aggfunc = 'count')
D1_count_investment_sector = D1_count_investment_sector.rename(columns = {'raised_amount_usd' : 'count_investment_sector'})
D1_sum_investment_sector = D1.pivot_table(values =  'raised_amount_usd',index = 'main_sector',aggfunc = 'sum')
D1_sum_investment_sector = D1_sum_investment_sector.rename(columns ={'raised_amount_usd' : 'sum_investment_sector'})

In [29]:
#merging to D1 
D1 = pd.merge(D1,D1_count_investment_sector,how = 'inner',on = 'main_sector')
D1 = pd.merge(D1,D1_sum_investment_sector,how = 'inner', on = 'main_sector')

In [30]:
#creating a new data frame D1 and counting total investment and sum of investment
D2 = master_frame.loc[(master_frame.funding_round_type == 'venture') & (master_frame.country_code == 'GBR') &(master_frame.raised_amount_usd >= 5000000) & (master_frame.raised_amount_usd <= 15000000)]
D2['raised_amount_usd'] = D2['raised_amount_usd'].apply(lambda x: x/1000000)
D2['raised_amount_usd'] = D2['raised_amount_usd'].apply(lambda x: round(x,2))

In [31]:
#now calculating sum of investment and no of investment in main sectors and renaming col
D2_count_investment_sector = D2.pivot_table(values = 'raised_amount_usd',index = 'main_sector',aggfunc = 'count')
D2_count_investment_sector = D2_count_investment_sector.rename(columns = {'raised_amount_usd' : 'count_investment_sector'})
D2_sum_investment_sector = D2.pivot_table(values =  'raised_amount_usd',index = 'main_sector',aggfunc = 'sum')
D2_sum_investment_sector = D2_sum_investment_sector.rename(columns ={'raised_amount_usd' : 'sum_investment_sector'})

In [32]:
#merging to D2
D2 = pd.merge(D2,D2_count_investment_sector,how = 'inner',on = 'main_sector')
D2 = pd.merge(D2,D2_sum_investment_sector,how = 'inner', on = 'main_sector')

In [33]:
#creating a new data frame D3
D3 = master_frame.loc[(master_frame.funding_round_type == 'venture') & (master_frame.country_code == 'IND') &(master_frame.raised_amount_usd >= 5000000) & (master_frame.raised_amount_usd <= 15000000)]
D3['raised_amount_usd'] = D3['raised_amount_usd'].apply(lambda x: x/1000000)
D3['raised_amount_usd'] = D3['raised_amount_usd'].apply(lambda x: round(x,2))

In [34]:
#now calculating sum of investment and no of investment in main sectors and renaming col
D3_count_investment_sector = D3.pivot_table(values = 'raised_amount_usd',index = 'main_sector',aggfunc = 'count')
D3_count_investment_sector = D3_count_investment_sector.rename(columns = {'raised_amount_usd' : 'count_investment_sector'})
D3_sum_investment_sector = D3.pivot_table(values =  'raised_amount_usd',index = 'main_sector',aggfunc = 'sum')
D3_sum_investment_sector = D3_sum_investment_sector.rename(columns ={'raised_amount_usd' : 'sum_investment_sector'})

In [35]:
#merging to D3
D3 = pd.merge(D3,D3_count_investment_sector,how = 'inner',on = 'main_sector')
D3 = pd.merge(D3,D3_sum_investment_sector,how = 'inner', on = 'main_sector')

In [36]:
#total no.of investment in country1
D1_count_investment_sector.sum()

count_investment_sector    12063
dtype: int64

In [37]:
# total amount of investment in country1
D1_sum_investment_sector.sum()

sum_investment_sector    107757.2
dtype: float64

In [38]:
#total no. of investment in country2
D2_count_investment_sector.sum()

count_investment_sector    621
dtype: int64

In [39]:
#total amount of investment in country 2
D2_sum_investment_sector.sum()

sum_investment_sector    5379.07
dtype: float64

In [40]:
# total no. of investment in country3
D3_count_investment_sector.sum()

count_investment_sector    328
dtype: int64

In [41]:
#total amount of investment in country3
D3_sum_investment_sector.sum()

sum_investment_sector    2949.55
dtype: float64

In [42]:
#top sector no. of investment wise in country1
D1_count_investment_sector.sort_values(by = 'count_investment_sector',ascending = False)

,count_investment_sector
main_sector,
Others,2950
"Social, Finance, Analytics, Advertising",2714
Cleantech / Semiconductors,2350
"News, Search and Messaging",1583
Health,909
Manufacturing,799
Entertainment,591
Automotive & Sports,167


In [43]:
#top sector no. of investment wise in country2
D2_count_investment_sector.sort_values(by = 'count_investment_sector',ascending = False)

,count_investment_sector
main_sector,
Others,147
"Social, Finance, Analytics, Advertising",133
Cleantech / Semiconductors,130
"News, Search and Messaging",73
Entertainment,56
Manufacturing,42
Health,24
Automotive & Sports,16


In [44]:
#top sector no. of investment wise in country 3
D3_count_investment_sector.sort_values(by = 'count_investment_sector', ascending = False)

,count_investment_sector
main_sector,
Others,110
"Social, Finance, Analytics, Advertising",60
"News, Search and Messaging",52
Entertainment,33
Manufacturing,21
Cleantech / Semiconductors,20
Health,19
Automotive & Sports,13


In [45]:
#company received highest investment in top sector in country1
D1_top_company_investment = D1.loc[(D1.main_sector == 'Others')]
D1_top_company_investment = D1_top_company_investment.pivot_table(values = 'raised_amount_usd',index = 'name',aggfunc = 'sum').sort_values(by = 'raised_amount_usd',ascending = False)
D1_top_company_investment

,raised_amount_usd
name,
/Organization/Virtustream,64.30
/Organization/Capella,54.97
/Organization/Airtight-Networks,54.20
/Organization/Decarta,52.10
/Organization/Black-Duck-Software,51.00
/Organization/Approva,50.10
/Organization/Five9,49.60
/Organization/Bit9,48.43
/Organization/Aryaka-Networks,48.17


In [46]:
#company received highest investment in top sector country2
D2_top_company_investment = D2.loc[(D2.main_sector == 'Others')]
D2_top_company_investment = D2_top_company_investment.pivot_table(values = 'raised_amount_usd', index = 'name', aggfunc = 'sum').sort_values(by = 'raised_amount_usd',ascending = False)
D2_top_company_investment.head()

,raised_amount_usd
name,
/Organization/Electric-Cloud,37.00
/Organization/Sensage,36.25
/Organization/Enigmatic,32.50
/Organization/Silverrail-Technologies,29.00
/Organization/Opencloud,27.97


In [47]:
#company received highest investment in top sector country3
D3_top_company_investment = D3.loc[(D3.main_sector == 'Others')]
D3_top_company_investment = D3_top_company_investment.pivot_table(values = 'raised_amount_usd', index = 'name', aggfunc = 'sum').sort_values(by = 'raised_amount_usd',ascending = False)
D3_top_company_investment.head()

,raised_amount_usd
name,
/Organization/Firstcry-Com,39.0
/Organization/Myntra,38.0
/Organization/Commonfloor,32.9
/Organization/Pepperfry-Com,28.0
/Organization/Itzcash-Card-Ltd,25.0


In [48]:
#company received highest investment in top  2nd sector country1
D1_top_company_investment_2nd = D1.loc[(D1.main_sector == 'Social, Finance, Analytics, Advertising')]
D1_top_company_investment_2nd = D1_top_company_investment_2nd.pivot_table(values = 'raised_amount_usd', index = 'name', aggfunc = 'sum').sort_values(by = 'raised_amount_usd',ascending = False)
D1_top_company_investment_2nd.head()

,raised_amount_usd
name,
/Organization/Shotspotter,67.93
/Organization/Demandbase,63.00
/Organization/Intacct,61.80
/Organization/Netbase,60.60
/Organization/Lotame,59.70


In [49]:
#company received highest investment in top  2nd sector country2
D2_top_company_investment_2nd = D2.loc[(D2.main_sector == 'Social, Finance, Analytics, Advertising')]
D2_top_company_investment_2nd = D2_top_company_investment_2nd.pivot_table(values = 'raised_amount_usd', index = 'name', aggfunc = 'sum').sort_values(by = 'raised_amount_usd',ascending = False)
D2_top_company_investment_2nd.head()

,raised_amount_usd
name,
/Organization/Celltick-Technologies,37.50
/Organization/Mythings,34.00
/Organization/Zopa,32.90
/Organization/Imagini,28.55
/Organization/Marketinvoice,25.55


In [50]:
#company received highest investment in top  2nd sector country3
D3_top_company_investment_2nd = D3.loc[(D3.main_sector == 'Social, Finance, Analytics, Advertising')]
D3_top_company_investment_2nd = D3_top_company_investment_2nd.pivot_table(values = 'raised_amount_usd', index = 'name', aggfunc = 'sum').sort_values(by = 'raised_amount_usd',ascending = False)
D3_top_company_investment_2nd.head()

,raised_amount_usd
name,
/Organization/Manthan-Systems,50.70
/Organization/Komli-Media,28.00
/Organization/Shopclues-Com,25.00
/Organization/Intarvo,21.90
/Organization/Grameen-Financial-Services,21.56
